In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, ForeignKey, Date
from sqlalchemy.orm import relationship
import pandas as pd

In [2]:
def load_lifts_sql(sql_db_file: str):
    """
    Loads lifts from SQL database to a DataFrame
    """
    engine = create_engine(f"sqlite:///{sql_db_file}", echo = True)
    
    df = pd.read_sql_table("lifts", engine)
    return df

df = load_lifts_sql("C:\\Development\\lifting-tracker\\lift_tracker.db")
df.head()

2021-01-03 14:52:28,393 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-01-03 14:52:28,397 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 14:52:28,399 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-01-03 14:52:28,400 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 14:52:28,402 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-01-03 14:52:28,403 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 14:52:28,405 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='view' ORDER BY name
2021-01-03 14:52:28,406 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 14:52:28,408 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("lifts")
2021-01-03 14:52:28,410 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 14:52:28,412 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL  

,id,exercise,category,weight,reps,orm,date,user_id
0,1,Seated Barbell Press,Shoulders,44.09,5.0,49.623683,2015-12-26,None
1,2,Seated Barbell Press,Shoulders,44.09,5.0,49.623683,2015-12-26,None
2,3,Seated Barbell Press,Shoulders,88.18,5.0,99.247367,2015-12-26,None
3,4,Seated Barbell Press,Shoulders,88.18,5.0,99.247367,2015-12-26,None
4,5,Seated Barbell Press,Shoulders,88.18,5.0,99.247367,2015-12-26,None


In [ ]:
def load_lifts_csv(csv_file: str) -> pd.DataFrame:
    """
    Load lifts .csv file to a DataFrame
    """
    # fields: date, exercise, category, weight, reps
    df = pd.read_csv(csv_file)

    # drop useless columns, change column types, etc.
    df = df.drop("Distance", 1)
    df = df.drop("Distance Unit", 1)
    df = df.drop("Time", 1)
    df["Date"] = pd.to_datetime(df["Date"])
    # df["Comment"] = df["Comment"].astype("object")
    df = df.drop("Comment", 1)

    # create 1RM column
    df["1RM"] = df["Weight (lbs)"] * 1.03 ** (df["Reps"] - 1)
    
    df = df.rename(columns = {"Weight (lbs)": "weight", "Date": "date", "Exercise": "exercise", "Category": "category", "Weight": "weight", "Reps": "reps", "1RM": "orm"})

    return df

df = load_lifts_csv(
        r"C:\Users\andre\Downloads\FitNotes_Export_2019_12_28_14_11_12.csv"
    )

df.head(10)

In [ ]:

engine = create_engine("sqlite:///C:\\Development\\lifting-tracker\\lift_tracker.db", echo = True)

Base = declarative_base()

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    username = Column("username", String(32))
    lifts = relationship("Lift")

class Lift(Base):
    __tablename__ = "lifts"
    id = Column(Integer, primary_key=True)
    exercise = Column("exercise", String(64))
    category = Column("category", String(32))
    weight = Column("weight", Numeric)
    reps = Column("reps", Integer)
    orm = Column("orm", Numeric)
    date = Column("date", Date)
    user_id = Column(Integer, ForeignKey("users.id"))
    
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker
import datetime
Session = sessionmaker(bind = engine)
session = Session()

In [ ]:
df.to_sql("lifts", engine, if_exists = "append", index = False)

In [ ]:
c1 = Lift(exercise = "Flat Barbell Bench Press", category = "Chest", weight = 290, reps = 1, orm = "290", date = datetime.date(2020, 12, 29), user_id = 1)

session.add(c1)
session.commit()

In [ ]:
result = session.query(Lift).first()

for row in result:
    print("Name: ",row.exercise, "Weight:",row.weight, "Reps:",row.reps)

In [ ]:
session.close()

In [ ]:
session.query(Lift).delete()

In [ ]:
db_cols = Lift.__table__.columns.keys()
df_cols = df.columns.tolist()

set(db_cols) - set(df_cols)